In [1]:
import requests
import json
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import translators as ts          
from transliterate import translit, get_available_language_codes
import time
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
import sqlite3
import ast
import re
import string
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point

import spacy
nlp = spacy.load('en_core_web_sm')
import seaborn as sns
from multiprocessing import Pool, cpu_count

import translators as ts          
from transliterate import translit, get_available_language_codes

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

# Download the stopwords from NLTK
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

Using region Virginia server backend.



## Set Variables

In [2]:
def pull_geonames(lat, lon, radius, username):
    url = f"http://api.geonames.org/findNearbyPlaceNameJSON?lat={lat}&lng={lon}&radius={radius}&username={username}"
    url2 = f"http://api.geonames.org/findNearbyJSON?lat={lat}&lng={lon}&radius={radius}&username={username}"

    try:
        response = requests.get(url)
        data = response.json()
    except:
        pass
    try:
        response2 = requests.get(url2)
        data2 = response2.json()
    except:
        pass
    
    return data, data2

## Create Nearby Place Names dataframe

In [3]:
def format_place_names(data):
    # Create a dataframe from the 'geonames' field in the JSON data
    df = pd.DataFrame(data['geonames'])
    df['lng'] = df['lng'].astype(float)
    df['lat'] = df['lat'].astype(float)
    df['type'] = ['Nearby Name' for x in range(len(df))] 

    return df

## Create Nearby Toponym dataframe

In [4]:
def format_topo_names(data2):
    # Create a dataframe from the 'geonames' field in the JSON data
    df2 = pd.DataFrame(data2['geonames'])

    df2['lng'] = df2['lng'].astype(float)
    df2['lat'] = df2['lat'].astype(float)
    df2['type'] = ['Nearby Toponym' for x in range(len(df2))] 

    return df2

## Query using the OSM Overpass API - Points of Interst

In [5]:
def pull_OSM_names(lat, lon, radius):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    node
      ["name"]
      (around:{radius},{lat},{lon});
    out;
    """

    response = requests.get(overpass_url, params={'data': overpass_query})
    data3 = response.json()
    
    return data3

In [6]:
def pull_OSM_streets(lat, lon, radius):
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    way
      ["highway"]["name"]
      (around:{radius},{lat},{lon});
    out tags;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()

    # Create a dataframe from the 'elements' field in the JSON data
    df = pd.json_normalize(data['elements'])

     # Rename columns with list
    cols = [x.replace('tags.', '') for x in df.columns]
    df.set_axis(cols, axis=1,inplace=True)

    # Now 'df' is a DataFrame where each row represents a street and columns represent fields like 'id', 'tags', 'street_name', etc.
    return df

In [7]:
def format_osm_names(data3):
    df4 = pd.json_normalize(data3['elements'])
    # Rename columns with list
    cols = [x.replace('tags.', '') for x in df4.columns]
    df4.set_axis(cols, axis=1,inplace=True)
    translit_names = []
    for n in df4['name']:
        try:
            translit_names.append(translit(u"{}".format(n), reversed=True))
        except:
            translit_names.append(n)
    df4['translit_name'] = translit_names  
    df4['type'] = ['OSM' for x in range(len(df4))]
    df4 = df4.rename(columns={'lon':'lng'})
    
    return df4


In [8]:
def subset_and_combine_data(df, df2, df4):
    df3 = pd.concat([df, df2]).reset_index(drop=True)
    df3_subset = df3[['lng', 'lat', 'name']]
    df4_subset = df4[['lng', 'lat', 'translit_name']]
    df4_subset.rename(columns={'translit_name':'name'}, inplace=True) 
    all_comb = pd.concat([df3_subset, df4_subset]).reset_index(drop=True)
    
    return all_comb  

# Develop a Gazetteer and augment with Conflict Indicative Words - 10km from events
- For each event, we assume there is a list of geonames w/in proximity of event
- We also assume conflict indicative words w/in the event notes

In [9]:
ACLED = R'D:\PENNSTATE_GIS\CAPSTONE!\ACLED\Europe-Central-Asia_2018-2023_Jan13.csv'
df = pd.read_csv(ACLED, engine='python', encoding='utf-8') 
ukr = df[df['COUNTRY'] == 'Ukraine']
ukr_2022 = ukr[ukr['YEAR'] == 2022].reset_index(drop=True)
ukr_2022.shape

(36094, 29)

In [10]:
# More interested in GEO Precision 1 or 2, as 3 is very generalized. 
acled_geo_1 = ukr_2022[ukr_2022['GEO_PRECISION']==1].reset_index(drop=True)
acled_geo_1['EVENT_DATE'] = pd.to_datetime(acled_geo_1['EVENT_DATE'], errors='coerce') 
acled_geo_1['MONTH'] = [x.month for x in acled_geo_1['EVENT_DATE']]
acled_geo_2 = ukr_2022[ukr_2022['GEO_PRECISION']==2].reset_index(drop=True)
acled_geo_2['EVENT_DATE'] = pd.to_datetime(acled_geo_2['EVENT_DATE'], errors='coerce') 
acled_geo_2['MONTH'] = [x.month for x in acled_geo_2['EVENT_DATE']]

In [11]:
summer_2022_geo1 = acled_geo_1[acled_geo_1['MONTH'].isin([6, 7, 8])].reset_index(drop=True)
summer_2022_geo1['GEONAMES_OSM'] = ""
summer_2022_geo1['CIW'] = ""
summer_2022_geo1.shape

(5107, 32)

## Append gazetteer place names to each event within 10km radius
- Transliterate Place name (not all places have an english name variant)

In [ ]:
geonames_osm = []
osm_streets = []
all_placeses = pd.DataFrame()
limit_met_at = ""
error_log_gn = ""
error_log_osm = []
error_log_streets = []
limit = False

for i,row in summer_2022_geo1.iterrows():
    display(f"Running index {i}...")
    lat = row.LATITUDE
    lon = row.LONGITUDE
    try: #-------------------GEONAMES PLACES/TOPO
        if not limit:
            radius = '10'
            username = 'ellerd1'

            data, data2 = pull_geonames(lat, lon, radius, username)

            df = format_place_names(data)
            df2 = format_topo_names(data2)
        else:
            pass
    except:
        limit_met_at = "GeoNames limit met at index " + str(i)
        error_log_gn = str(data)
        limit = True
        print("GeoNames error on index ", i)
        pass
    try: #-------------------OSM POIS
        radius = '10000'
        data3 = pull_OSM_names(lat, lon, radius)
        df4 = format_osm_names(data3)
    except:
        error_log_osm.append(f"OSM streets error on index {i}")
        print("OSM error on index ", i)
        df4 = pd.DataFrame()
        pass
    
    try: #-------------------OSM STREETS
        radius = '10000'
        df5 = pull_OSM_streets(lat, lon, radius)
        df5.dropna(subset=['name'], inplace=True)
        translit_names = []
        for n in df5['name']:
            try:
                translit_names.append(translit(u"{}".format(n), reversed=True))
            except:
                translit_names.append(n)
        df5['translit_name'] = translit_names            
        df5['type'] = ['OSM-S' for x in range(len(df5))]
        df5 = df5.rename(columns={'lon':'lng'})
        osm_streets.append(df5['translit_name'].to_list())
        
    except:
        error_log_streets.append(f"OSM streets error on index {i}")
        print("OSM streets error on index ", i)
        print(lat, lon)
        osm_streets.append([])
        break
        
    if limit:
        if len(df4) !=0:
                # Only adds OSM names if GeoNames limit met
            all_comb = df4[['lng', 'lat', 'translit_name']]
            all_comb.rename(columns={'translit_name':'name'}, inplace=True)
            all_placeses = all_placeses.append(all_comb)

        else:
            all_placeses = all_placeses.append(df4)
    else:
        print('here')
            # Combines if GeoNames limit not met
        all_comb = subset_and_combine_data(df, df2, df4)
        all_placeses = all_placeses.append(all_comb)

    geonames_osm.append(all_comb['name'].to_list())
    
    clear_output(wait=True)
       

In [ ]:
# summer_2022_geo1['GEONAMES_OSM'] = geonames_osm
# summer_2022_geo1['STREETS'] = osm_streets

In [ ]:
# # Convert to datatypes that SQlite likes.
# summer_2022_geo1['EVENT_DATE'] = summer_2022_geo1['EVENT_DATE'].astype(str)
# summer_2022_geo1['GEONAMES_OSM'] = summer_2022_geo1['GEONAMES_OSM'].astype(str)
# summer_2022_geo1['STREETS'] = summer_2022_geo1['STREETS'].astype(str)
# summer_2022_geo1.fillna(value="")

## Store ACLED 2022 Summer Conflict Events with places in SQlite

In [12]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\ACLED_summer_2022_geo1.db')
cursor = conn.cursor()

In [13]:
# %%time
# summer_2022_geo1.to_sql('ACLED_SUMMER_2022', conn, if_exists='append', index=False)

In [14]:
query = """SELECT * from ACLED_SUMMER_2022"""
df_acled = pd.read_sql(query, conn)
df_acled['GEONAMES_OSM'] = df_acled['GEONAMES_OSM'].apply(ast.literal_eval)
df_acled['STREETS'] = df_acled['STREETS'].apply(ast.literal_eval)
display(df_acled.head())
display(df_acled.shape)

,ISO,EVENT_ID_CNTY,EVENT_ID_NO_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,...,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES,TIMESTAMP,MONTH,GEONAMES_OSM,CIW,STREETS
0,804,UKR59164,59164,2022-06-01,2022,1,Battles,Armed clash,Military Forces of Russia (2000-),None,...,1,Novoye Vremya Ukraine; Ministry of Defence of ...,Other-National,"On 1 June 2022, Russian forces engaged in an o...",0,1654627711,6,"[Kudrjashіvka, Purdіvka, Voєvodіvka, Sinets'ki...",,"[Generala Potapenka vulitsja, Proletars'ka vul..."
1,804,UKR59197,59197,2022-06-01,2022,1,Battles,Armed clash,Military Forces of Ukraine (2019-),None,...,1,Focus; 24 Channel,National,"On 1 June 2022, Ukrainian forces killed a Lieu...",41,1654627711,6,"[Stavki, Brusіvka, Liman, Schurove, Starij Kar...",,"[Komarova vulitsja, Gogolja vulitsja, Kizima v..."
2,804,UKR59498,59498,2022-06-01,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,Novoye Vremya Ukraine; 24 Channel; Ministry of...,Other-National,"On 1 June 2022, Russian forces fired 152mm mor...",2,1654627711,6,"[Mikolaїv, Jakova Butovicha vulitsja, Ochakіvs...",,"[Nіkol's'ka vulitsja, Odes'ke shose, Geroїv Uk..."
3,804,UKR59250,59250,2022-06-01,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,24 Channel,National,"On 1 June 2022, Russian forces shelled Kharkiv...",1,1654627711,6,"[Harkіv - Nova Bavarіja, Novoselіvka, Unіversi...",,"[Chkalova vulitsja, 1-j Chobotars'kij v’їzd, V..."
4,804,UKR59501,59501,2022-06-01,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,24 Channel,National,"On 1 June 2022, Russian forces fired BM-21 Gra...",0,1654627711,6,"[Ochakіvs'kij mis, Ochakіv, Osetrіvka, Іvanіvk...",,"[Ol'vіjs'ka vulitsja, 8-go Bereznja vulitsja, ..."


(5107, 33)

In [143]:
    # Average length of 10km surrounding names from GeoNames/OSM lists
sum([len(l) for l in df_acled['GEONAMES_OSM'].to_list()]) / len(df_acled['GEONAMES_OSM'])

603.999608380654

In [144]:
    # Average length of 10km surrounding names from GeoNames/OSM lists
sum([len(l) for l in df_acled['STREETS'].to_list()]) / len(df_acled['STREETS'])

729.8202467201879

In [15]:
conn.close()

## Store GeoNames_OSM place coordinates in SQlite

In [18]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\osm_place_coords_lookup.db')
cursor = conn.cursor()

In [19]:
# all_placeses.to_sql('place_coords_lookup', conn, if_exists='append', index=False)

In [20]:
query = """SELECT * from place_coords_lookup"""
place_coords_df = pd.read_sql(query, conn)
display(place_coords_df.head())
display(place_coords_df.shape)

,lng,lat,name
0,38.451080,49.031597,Kudrjashіvka
1,38.579138,48.977480,Purdіvka
2,38.444580,48.980465,Voєvodіvka
3,38.411372,48.948004,Sinets'kij
4,38.493648,48.947870,Sєvєrodonets'k


(3084567, 3)

In [21]:
conn.commit()
conn.close()

## Data Preprocessing - Data Cleaning
- clean the text data, remove punctuation, convert all text to lower case, etc.

In [146]:
def clean_text(text):
        #Make text lowercase   
    text = text.lower()
        #remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)
        #remove punctuation   
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
        #remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
        #remove links   
    text = re.sub(r'http\S+', '', text)
        #remove emojis    
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    text = regrex_pattern.sub(r'', text)
    
    return text

In [147]:
%%time
clean_reports = df_acled.NOTES.apply(lambda x: clean_text(x))
clean_reports = clean_reports.to_list()
display(clean_reports[0])

'on  june  russian forces engaged in an offensive in severodonetsk luhansk russian forces occupied  of the city ukrainian forces conducted counter attacks and captured six russian soldiers casualties unknown'

Wall time: 94 ms


### Subset into Event Type - for below NLP practices

In [148]:
battles_df = df_acled[df_acled['EVENT_TYPE'] == 'Battles']
explosions_rv_df = df_acled[df_acled['EVENT_TYPE'] == 'Explosions/Remote violence']
strategic_dev_df = df_acled[df_acled['EVENT_TYPE'] == 'Strategic developments']
protests_df = df_acled[df_acled['EVENT_TYPE'] == 'Protests']
civ_violence_df = df_acled[df_acled['EVENT_TYPE'] == 'Violence against civilians']

In [149]:
clean_reports_battles = battles_df.NOTES.apply(lambda x: clean_text(x))
clean_reports_battles = clean_reports_battles.to_list()
clean_reports_battles_doc = ' '.join(clean_reports_battles)

clean_reports_explosions = explosions_rv_df.NOTES.apply(lambda x: clean_text(x))
clean_reports_explosions = clean_reports_explosions.to_list()
clean_reports_explosions_doc = ' '.join(clean_reports_explosions)

clean_reports_strategic = strategic_dev_df.NOTES.apply(lambda x: clean_text(x))
clean_reports_strategic = clean_reports_strategic.to_list()
clean_reports_strategic_doc = ' '.join(clean_reports_strategic)

clean_reports_protests = protests_df.NOTES.apply(lambda x: clean_text(x))
clean_reports_protests = clean_reports_protests.to_list()
clean_reports_protests_doc = ' '.join(clean_reports_protests)

clean_reports_civ = civ_violence_df.NOTES.apply(lambda x: clean_text(x))
clean_reports_civ = clean_reports_civ.to_list()
clean_reports_civ_doc = ' '.join(clean_reports_civ)


In [150]:
event_type_documents_combined = [clean_reports_battles_doc, clean_reports_explosions_doc, clean_reports_strategic_doc, clean_reports_protests_doc, clean_reports_civ_doc]

## Find the most relevant words across all reports.
- Calculate the TF-IDF matrix for all the events, then find the words with the highest average TF-IDF score across all documents.

In [155]:
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]

# create the transform
vectorizer = TfidfVectorizer(tokenizer=lemmatize_text, stop_words=stopwords.words('english'))

In [156]:
# tokenize and build vocabulary
vectorizer.fit(clean_reports)

# Transform the reports to a matrix of TF-IDF features
tfidf_matrix = vectorizer.transform(clean_reports)

# Convert the matrix to a DataFrame for easier manipulation
df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Calculate the mean TF-IDF score for each word across all documents
mean_scores = df.mean()

# Sort the words by their mean score in descending order
sorted_scores = mean_scores.sort_values(ascending=False)

# Print the words with the highest mean TF-IDF score between all documents
sorted_scores.to_frame().head(25)

,0
force,0.096541
casualty,0.094852
unknown,0.094339
russian,0.089324
shelled,0.086228
donetsk,0.085281
artillery,0.066312
june,0.065278
july,0.063422
august,0.062320


In [86]:
# This result is a matrix where each row corresponds to an event 'document'
# and each column corresponds to a word in the 'vocabulary', 
# and the value is the TF-IDF score of that word in that document.
df

,abakumov,abandoned,abducted,abused,access,accidentally,according,accuse,accused,accusing,...,zolotarivka,zolote,zone,zoo,zoria,zorya,zoryane,zrubanka,zvanivka,zymohiria
0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5103,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5104,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5105,0.0,0.0,0.30516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
# DataFrame where the index is the event(document) number and the columns are the ranks from 1 to N,  
# suggesting that the higher ranked words are important in distinguishing each particular document from the others

N = 35
top_N = pd.DataFrame({n: df.T[col].nlargest(N).index.tolist() 
                      for n, col in enumerate(df.T)}).T
top_N.index = ["document_" + str(i) for i in range(len(clean_reports))]

top_N

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
document_0,counter,captured,six,engaged,offensive,occupied,severodonetsk,city,soldier,attack,...,accuse,accused,accusing,across,action,active,actively,activist,activity,actor
document_1,colonel,lieutenant,battalion,chechen,lyman,soldier,killed,ukrainian,june,donetsk,...,actively,activist,activity,actor,adamivka,additionally,adjacent,administration,administrative,admitted
document_2,two,civilian,killing,wounded,mykolaiv,infrastructure,mortar,fired,june,russian,...,activist,activity,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption
document_3,woman,shelling,result,one,killed,wa,kharkiv,june,shelled,russian,...,activist,activity,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption
document_4,ochakiv,grad,missile,mykolaiv,fired,june,russian,unknown,casualty,force,...,activity,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption,advance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
document_5102,toretsk,region,august,donetsk,shelled,russian,unknown,casualty,force,abakumov,...,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption,advance,advanced
document_5103,pivdenne,injured,wa,region,civilian,august,donetsk,shelled,russian,force,...,activity,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption,advance
document_5104,heorhiivka,killed,wa,region,civilian,august,donetsk,shelled,russian,force,...,activity,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption,advance
document_5105,kakhovka,presumably,whereabouts,resident,abducted,kherson,august,russian,unknown,force,...,actor,adamivka,additionally,adjacent,administration,administrative,admitted,adoption,advance,advanced


### The script above appends words NOT in the document after it reaches its limit of entities in the doc

### Attach report relevant words - CIW

In [103]:
    # Check if records are not long enough to account or all sorted TF-IDF words
for i,r in top_N.iterrows():
    l = top_N.iloc[int(i.split('_')[1])].to_list()
    if 'abakumov' not in l:
        x = clean_reports[int(i.split('_')[1])]
        
        print(len(x.replace("  ", " ").split(' ')))    

In [122]:
    # Attach list relevant to the size of the report
for i,r in top_N.iterrows():
    if i == 'document_4828':
        ind = int(i.split('_')[1])
        l = top_N.iloc[ind].to_list()
        ciws = ' '.join(l).split('abandoned')[0][:-1].split(' ')
        df_acled.at[ind, 'CIW'] = ciws

    else:
        ind = int(i.split('_')[1])
        l = top_N.iloc[ind].to_list()
        ciws = ' '.join(l).split('abakumov')[0][:-1].split(' ')
        df_acled.at[ind, 'CIW'] = ciws

In [129]:
print(df_acled.iloc[5].CIW)

['night', 'slovyansk', 'least', 'attack', 'damaging', 'building', 'residential', 'launched', 'rocket', 'june', 'donetsk', 'russian', 'casualty', 'force']


In [130]:
df_acled.iloc[5].NOTES

'On 1 June 2022, at night, Russian forces launched a rocket attack on Slovyansk, Donetsk, damaging at least 10 residential buildings. There were no casualties.'

In [131]:
df_acled

,ISO,EVENT_ID_CNTY,EVENT_ID_NO_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,...,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES,TIMESTAMP,MONTH,GEONAMES_OSM,CIW,STREETS
0,804,UKR59164,59164,2022-06-01,2022,1,Battles,Armed clash,Military Forces of Russia (2000-),None,...,1,Novoye Vremya Ukraine; Ministry of Defence of ...,Other-National,"On 1 June 2022, Russian forces engaged in an o...",0,1654627711,6,"[Kudrjashіvka, Purdіvka, Voєvodіvka, Sinets'ki...","[counter, captured, six, engaged, offensive, o...","[Generala Potapenka vulitsja, Proletars'ka vul..."
1,804,UKR59197,59197,2022-06-01,2022,1,Battles,Armed clash,Military Forces of Ukraine (2019-),None,...,1,Focus; 24 Channel,National,"On 1 June 2022, Ukrainian forces killed a Lieu...",41,1654627711,6,"[Stavki, Brusіvka, Liman, Schurove, Starij Kar...","[colonel, lieutenant, battalion, chechen, lyma...","[Komarova vulitsja, Gogolja vulitsja, Kizima v..."
2,804,UKR59498,59498,2022-06-01,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,Novoye Vremya Ukraine; 24 Channel; Ministry of...,Other-National,"On 1 June 2022, Russian forces fired 152mm mor...",2,1654627711,6,"[Mikolaїv, Jakova Butovicha vulitsja, Ochakіvs...","[two, civilian, killing, wounded, mykolaiv, in...","[Nіkol's'ka vulitsja, Odes'ke shose, Geroїv Uk..."
3,804,UKR59250,59250,2022-06-01,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,24 Channel,National,"On 1 June 2022, Russian forces shelled Kharkiv...",1,1654627711,6,"[Harkіv - Nova Bavarіja, Novoselіvka, Unіversi...","[woman, shelling, result, one, killed, wa, kha...","[Chkalova vulitsja, 1-j Chobotars'kij v’їzd, V..."
4,804,UKR59501,59501,2022-06-01,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,24 Channel,National,"On 1 June 2022, Russian forces fired BM-21 Gra...",0,1654627711,6,"[Ochakіvs'kij mis, Ochakіv, Osetrіvka, Іvanіvk...","[ochakiv, grad, missile, mykolaiv, fired, june...","[Ol'vіjs'ka vulitsja, 8-go Bereznja vulitsja, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,804,UKR70833,70833,2022-08-31,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,Ukrinform,National,"On 31 August 2022, Russian forces shelled Tore...",0,1663096259,8,"[Torets'k, Dilіїvka, Druzhba, Pіvnіchne, Dachn...","[toretsk, region, august, donetsk, shelled, ru...","[Stantsіja Majors'ka vulitsja, Miru vulitsja, ..."
5103,804,UKR70799,70799,2022-08-31,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,Ukrinform,National,"On 31 August 2022, Russian forces shelled Pivd...",0,1663096259,8,"[Gorlіvka, Zajtseve, Shiroka Balka, Torets'k, ...","[pivdenne, injured, wa, region, civilian, augu...","[vulitsja Marshala Peresipkіna, vulitsja Marsh..."
5104,804,UKR70722,70722,2022-08-31,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),None,...,1,Ukrinform,National,"On 31 August 2022, Russian forces shelled Heor...",1,1663096259,8,"[Gostre, Kurahіvka, Vovchenka, Georgіїvka, Mak...","[heorhiivka, killed, wa, region, civilian, aug...","[Amudar’їns'ka vulitsja, Kalіnіngrads'ka vulit..."
5105,804,UKR72216,72216,2022-08-31,2022,1,Violence against civilians,Abduction/forced disappearance,Military Forces of Russia (2000-),None,...,1,Flot 2017,National,"On 31 August 2022, Russian forces abducted a r...",0,1663703413,8,"[Shljahove, Zmіїvka, Novoberislav, Berislav, L...","[kakhovka, presumably, whereabouts, resident, ...","[Dnіprovs'kij prospekt, Promislova vulitsja, 8..."


In [134]:
# Connect to the SQLite database
conn = sqlite3.connect(r'D:\PENNSTATE_GIS\CAPSTONE!\sql_db_lite\ACLED_summer_2022_geo_ciw.db')
cursor = conn.cursor()

In [137]:
# %%time
# # Convert to datatypes that SQlite likes.
# df_acled['GEONAMES_OSM'] = df_acled['GEONAMES_OSM'].astype(str)
# df_acled['CIW'] = df_acled['CIW'].astype(str)
# df_acled['STREETS'] = df_acled['STREETS'].astype(str)

# df_acled.to_sql('ACLED_summer_2022_geo_ciw', conn, if_exists='append', index=False)

5107

## Find the most relevant words between Event Types using TF-IDF

In [157]:
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]

# create the transform
vectorizer = TfidfVectorizer(tokenizer=lemmatize_text, stop_words=stopwords.words('english'))

In [158]:
# tokenize and build vocabulary
vectorizer.fit(event_type_documents_combined)

# Transform the reports to a matrix of TF-IDF features
tfidf_matrix = vectorizer.transform(event_type_documents_combined)

# Convert the matrix to a DataFrame for easier manipulation
df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Calculate the mean TF-IDF score for each word across all documents
mean_scores = df.mean()

# Sort the words by their mean score in descending order
sorted_scores = mean_scores.sort_values(ascending=False)

# Print the words with the highest mean TF-IDF score between all documents
sorted_scores.to_frame().head(15)

,0
force,0.347458
russian,0.312827
ukrainian,0.193185
casualty,0.172580
unknown,0.159071
august,0.150698
region,0.136848
july,0.127162
kherson,0.116568
june,0.116499


In [35]:
# DataFrame where the index is the event(document) number and the columns are the ranks from 1 to N,  
# suggesting that the higher ranked words are important in distinguishing each particular document from the others

N = 20
top_N = pd.DataFrame({n: df.T[col].nlargest(N).index.tolist() 
                      for n, col in enumerate(df.T)}).T
top_N.index = [i for i in ['battles_doc', 'explosions_doc', 'strategic_doc', 'protests_doc', 'violence_doc']]

top_N

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
battles_doc,force,casualty,russian,unknown,ukrainian,clash,july,donetsk,region,following,june,shelled,clashed,control,took,luhansk,artillery,severodonetsk,offensive,clashing
explosions_doc,casualty,shelled,force,unknown,russian,artillery,fired,donetsk,july,june,civilian,august,ukrainian,position,dpr,kharkiv,mortar,infrastructure,region,wa
strategic_doc,russian,force,looting,interception,kherson,ukrainian,region,july,june,august,shot,displacement,evacuated,stole,around,property,destruction,resident,looted,civilian
protests_doc,protest,relative,gathered,pow,august,ukrainian,attention,defender,russian,azovstal,captive,kyiv,olenivka,draw,peaceful,response,return,held,member,situation
violence_doc,abducted,russian,force,kherson,civilian,region,august,tortured,wa,around,detained,kharkiv,june,illegally,ukrainian,july,month,unknown,izium,released


## Apply Named Entity Recognition (NER) identifying the most common location entities in conflict reports.
- Then look for overlaps in keywords, topics or extracted entities between the tweets and the conflict event categories.

In [37]:
def extract_entitites(input: list) -> list:
    events_slice=[]
    for text in tqdm(input):
        doc = nlp(text)
        for ent in doc.ents: # This doc property is used for the named entities in the document. 
        #If the entity recognizer has been applied, this property will return a tuple of named entity span objects.
            events_slice += [[ent.text, ent.label_,1]]
    return events_slice

In [38]:
ne_data=pd.DataFrame(data=extract_entitites(clean_reports),columns=['named_entity','label','count'])
ne_data_grouped=ne_data.groupby(['named_entity','label'],as_index=False).sum()
display(ne_data_grouped)

100%|█████████████████████████████████████████████████████████████████████████████| 5107/5107 [00:24<00:00, 210.15it/s]


,named_entity,label,count
0,a few days,DATE,2
1,a week,DATE,1
2,abakumov,ORG,1
3,about tons,QUANTITY,1
4,about three days,DATE,1
...,...,...,...
557,zolotarivka luhansk region,PERSON,1
558,zoria,PERSON,1
559,zoria kherson,PERSON,1
560,zoria mykolaiv,PERSON,1


In [39]:
labels=ne_data_grouped['label'].unique().tolist()

for label in labels:
    display(label + " -- " + spacy.explain(label))

'DATE -- Absolute or relative dates or periods'

'ORG -- Companies, agencies, institutions, etc.'

'QUANTITY -- Measurements, as of weight or distance'

'NORP -- Nationalities or religious or political groups'

'PERSON -- People, including fictional'

...

'LOC -- Non-GPE locations, mountain ranges, bodies of water'

'CARDINAL -- Numerals that do not fall under another type'

'GPE -- Countries, cities, states'

'TIME -- Times smaller than a day'

'ORDINAL -- "first", "second", etc.'

In [40]:
gpe_label = ne_data_grouped[ne_data_grouped['label']=='GPE'].sort_values('count',ascending=False).reset_index(drop=True)
gpe_label_lst = gpe_label['named_entity'].to_list()
display(gpe_label.head(5))

,named_entity,label,count
0,new york,GPE,22
1,russia,GPE,19
2,luhansk,GPE,5
3,novosilka,GPE,4
4,stara hnylytsia,GPE,4


In [41]:
loc_label = ne_data_grouped[ne_data_grouped['label']=='LOC'].sort_values('count',ascending=False).reset_index(drop=True)
loc_label_lst = loc_label['named_entity'].to_list()
display(loc_label.head(5))

,named_entity,label,count
0,nova,LOC,40
1,kakhovka kherson region,LOC,2
2,nova kakhovka,LOC,2
3,antonivka kherson,LOC,1
4,atlantic,LOC,1


In [42]:
fac_label = ne_data_grouped[ne_data_grouped['label']=='FAC'].sort_values('count',ascending=False).reset_index(drop=True)
fac_label_lst = fac_label['named_entity'].to_list()
display(fac_label.head(5))

,named_entity,label,count
0,polyana,FAC,3
1,zelenyi hai mykolaiv casualties,FAC,2
2,the vuhlehirska tpp,FAC,2
3,stara hnylytsia,FAC,2
4,skadovsk kherson region casualties,FAC,2


## Create Mapbox GeoVisualization for Data Points Above

In [20]:
px.set_mapbox_access_token('pk.eyJ1IjoiZWxsZXJkMSIsImEiOiJjbDA3OGNhM2UwMHRlM2ZwOGlkNXQya3QwIn0.zl-zvxhtKHAx0fYPZsD4YQ')

In [23]:
fig = px.scatter_mapbox(df_acled, lat="LATITUDE", lon="LONGITUDE", color='EVENT_TYPE', labels={"EVENT_TYPE": "Event Type:"})

fig.update_layout(
    mapbox=dict(
#         style='mapbox://styles/mapbox/streets-v11',
        center=dict(lat=49.2, lon=32.0),
        zoom=5
    ),
    margin={"r": 0, "t": 40, "l": 0, "b": 0},  # Adjust the top margin to make space for the title
    height=600  # Adjust the height of the figure as needed
)

fig.update_layout(
    title={'text': 'All June, July, & August Armed Conflict Events by Event Type',
           'y': 0.98,  # Adjust the y position of the title
           'x': 0.5,
           'xanchor': 'center',
           'yanchor': 'top'}
)

fig.show()

### Had to place the below data in ArcPro, otherwise it crashes my notebook (Not enough RAM!)

In [45]:
# place_coords_df.to_csv(r'D:\PENNSTATE_GIS\CAPSTONE!\ACLED\event_radius_places.csv')

In [ ]:
# fig = px.scatter_mapbox(place_coords_df, lat="lat", lon="lng", labels={"name": "Place of Interest"})

# fig.update_layout(
#     mapbox=dict(
#         style='mapbox://styles/mapbox/streets-v11',
#         center=dict(lat=49.2, lon=32.0),
#         zoom=5
#     ),
#     margin={"r": 0, "t": 40, "l": 0, "b": 0},  # Adjust the top margin to make space for the title
#     height=600  # Adjust the height of the figure as needed
# )

# fig.update_layout(
#     title={'text': 'All GeoNames(limited) & OSM POIs within 10km of ACLED Events',
#            'y': 0.98,  # Adjust the y position of the title
#            'x': 0.5,
#            'xanchor': 'center',
#            'yanchor': 'top'}
# )

# fig.show()